<table style="font-size: 1em; padding: 0; margin: 0;">

<tr style="vertical-align: top; padding: 0; margin: 0;background-color: #ffffff">
        <td style="vertical-align: top; padding: 0; margin: 0; padding-right: 15px;">
    <p style="background: #182AEB; color:#ffffff; text-align:justify; padding: 10px 25px;">
        <strong style="font-size: 1.0em;"><span style="font-size: 1.2em;"><span style="color: #ffffff;">Deep Learning </span> for Satellite Image Classification (Manning Publications)</span><br/>by <em>Daniel Buscombe</em></strong><br/><br/>
        <strong>> Chapter 5: Model Optimization </strong><br/>
    </p>           
        
<p style="border: 1px solid #182AEB; border-left: 15px solid #182AEB; padding: 10px; text-align:justify;">
    <strong style="color: #182AEB">What you learned in Part 4.</strong>  
    <br/>In Part 4, you trained U-Net models to segment water pixels in imagery
    </p>
    
<p style="border: 1px solid #ff5733; border-left: 15px solid #ff5733; padding: 10px; text-align:justify;">
    <strong style="color: #ff5733">What you will learn in this Part.</strong>  
    <br/>In Part 5, you will optimize the models you made in Part 4. You will first learn how to tune hyper-parameters, add regularization and modify model architectures in several ways, and deal with class imbalance. Then, you will learn how to use a machine learning model known as a ‘fully-connected conditional random field” to refine label images, so labels are as accurate as possible.
    </p>    

#### Preliminaries for Colab

Like Part 3 and 4 previously, below are some convenience functions for those working on Google Colab with a GPU runtime

In [31]:
#colab = 0
colab = 1

In [ ]:
if colab==1:
    %tensorflow_version 2.x
    !pip install --default-timeout=1000 tensorflow-gpu==2.0   

You may have to restart the runtime and/or change runtime type here, if the following doesn't show Tensorflow version 2, and a GPU available

In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_gpu_available())

2.0.0
False


Convenience functions if you need to download example (minimal) imagery sets derived from NWPU and Sentinel-2 cloudless:

In [32]:
# from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


#s2 cloudless imagery
file_id = '1iMfIjr_ul49Ghs2ewazjCt8HMPfhY47h'
destination = 's2cloudless_imagery.zip'
if colab==1:
    download_file_from_google_drive(file_id, destination)

#s2 cloudless labels
file_id = '1c7MpwKVejoUuW9F2UaF_vps8Vq2RZRfR'
destination = 's2cloudless_label_imagery.zip'
if colab==1:
    download_file_from_google_drive(file_id, destination)

#nwpu imagery
file_id = '1gtuqy1VlU8-M5IEMnmiSuTlI5PxQPnGB'
destination = 'nwpu_images.zip'
if colab==1:
    download_file_from_google_drive(file_id, destination)

#nwpu labels
file_id = '1W5LGbcYAcFbG5YjLgX_ekBn0u5Rno35x'
destination = 'nwpu_label_images.zip'
if colab==1:
    download_file_from_google_drive(file_id, destination)                        

In [33]:
import zipfile
def unzip(f):
    """
    f = file to be unzipped
    """    
    with zipfile.ZipFile(f, 'r') as zip_ref:
        zip_ref.extractall()
        
if colab==1:
    unzip('s2cloudless_imagery.zip')
    unzip('s2cloudless_label_imagery.zip')   
    unzip('nwpu_images.zip')
    unzip('nwpu_label_images.zip')       

Import the libraries we will need:

In [13]:
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import Concatenate, Conv2DTranspose
from tensorflow.keras.models import Model
import numpy as np
import json, os
from random import shuffle
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

<table style="font-size: 1em; padding: 0; margin: 0;">

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #182AEB;">
    </small><br/>Training tuning strategies</h1>
<br/>
<p style="border-left: 15px solid #182AEB; text-align:justify; padding: 0 10px;">
We will set up a baseline model with no optimization using the NWPU imagery, then explore the following training optimization strategies:
<ul>
  <li>Building a bigger model with more layers</li>    
  <li>Using Early Stopping and Adaptive Learning Rates</li> 
  <li>Using a bigger model (and dropout)</li> 
  <li>Using regularization (Batch Normalization)</li> 
  <li>Using residual connections</li> 
</ul>
</p>
        </tr>
        </table>

#### Getting things set up with a baseline model
Zip through all these functions that we defined in the last Part. 

Like in the last Part, we will define the ```unet``` model, create model training callbacks, and generate augmented imagery

We use Keras callbacks to implement learning rate decay if the validation loss does not improve for 5 continues epochs. Called "reduce loss on plateau"

Also, we implement early stopping if the validation loss does not improve for 5 continuous epochs.

In [20]:
def image_batch_generator(files, batch_size = 32, sz = (512, 512)):
  
  while True: # this is here because it will be called repeatedly by the training function
    
    #extract a random subset of files of length "batch_size"
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs (x) and outputs (y)
    batch_x = []
    batch_y = []
    
    #cycle through each image in the batch
    for f in batch:

        #preprocess the raw images 
        rawfile = f'nwpu_images/data/{f}'
        raw = Image.open(rawfile)
        raw = raw.resize(sz)
        raw = np.array(raw)

        #check the number of channels because some of the images are RGBA or GRAY
        if len(raw.shape) == 2:
            raw = np.stack((raw,)*3, axis=-1)

        else:
            raw = raw[:,:,0:3]
            
        #get the image dimensions, find the min dimension, then square the image off    
        nx, ny, nz = np.shape(raw)
        n = np.minimum(nx,ny)
        raw = raw[:n,:n,:] 
            
        batch_x.append(raw)
        
        #get the masks. 
        maskfile = rawfile.replace('nwpu_images','nwpu_label_images')+'_mask.jpg'
        mask = Image.open(maskfile)
        # the mask is 3-dimensional so get the max in each channel to flatten to 2D
        mask = np.max(np.array(mask.resize(sz)),axis=2)
        # water pixels are always greater than 100
        mask = (mask>200).astype('int')
        
        mask = mask[:n,:n]

        batch_y.append(mask)

    #preprocess a batch of images and masks 
    batch_x = np.array(batch_x)/255. #divide image by 255 to normalize
    batch_y = np.array(batch_y)
    batch_y = np.expand_dims(batch_y,3) #add singleton dimension to batch_y

    yield (batch_x, batch_y) #yield both the image and the label together

We've seen code like the below in the previous Part, setting up batch size, proportion of the dataset to train with, getting randomized lists of test and train file names, and finally setting up generators for model training and testing

In [ ]:
batch_size = 8

prop_train = 0.6

all_files = os.listdir('nwpu_images/data')
shuffle(all_files)

split = int(prop_train * len(all_files))

#split into training and testing
train_files = all_files[0:split]
test_files  = all_files[split:]

train_generator = image_batch_generator(train_files, batch_size = batch_size)
test_generator  = image_batch_generator(test_files, batch_size = batch_size)

A customary check that things worked

In [ ]:
x, y = next(train_generator)
plt.imshow(x[0])
plt.imshow(y[0].squeeze(), cmap='gray', alpha=0.5)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [ ]:
# a tolerance for the training.
min_delta = 0.0001

# minimum learning rate (lambda)
min_lr = 0.0001

# the factor applied to the learning rate when the appropriate triggers are made
factor = 0.8

batch_size = 8

filepath = 'unet'+str(batch_size)+'.h5'

In [ ]:
train_generator = image_batch_generator(train_files, batch_size = batch_size)
test_generator  = image_batch_generator(test_files, batch_size = batch_size)
train_steps = len(train_files) //batch_size
test_steps = len(test_files) //batch_size
print(train_steps)
print(test_steps)

<table style="font-size: 1em; padding: 0; margin: 0;">

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #182AEB;">
    </small><br/>Optimization strategy: changing model architecture <br/>using residual connections</h1>
<br/>
<p style="border-left: 15px solid #182AEB; text-align:justify; padding: 0 10px;">
A standard approach is to pass the input image goes through multiple convolutions and obtain high-level features. In a network architecture with <em>residual layers</em> or connections, each layer gets to see both the output from the previous layer (standard) as well as the inputs to that layer. So it not only sees the ouputs but also the data used to learn that output
</p>
        </tr>
        </table>

![](https://cdn-images-1.medium.com/max/1000/1*4wx7szWCBse9-7eemGQJSw.png)

Import `Activation` and `Add` layers from keras

In [17]:
from tensorflow.keras.layers import Activation, Add

Create a new UNet model function. This time we'll use a few convenience functions

In [11]:
def batchnorm_act(x):
    x = BatchNormalization()(x)
    return Activation("relu")(x)

def conv_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = batchnorm_act(x)
    return Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)

def bottleneck_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    conv = conv_block(conv, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    
    bottleneck = Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    bottleneck = batchnorm_act(bottleneck)
    
    return Add()([conv, bottleneck])

def res_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    res = conv_block(x, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    res = conv_block(res, filters, kernel_size=kernel_size, padding=padding, strides=1)
    
    bottleneck = Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    bottleneck = batchnorm_act(bottleneck)
    
    return Add()([bottleneck, res])

def upsamp_concat_block(x, xskip):
    u = UpSampling2D((2, 2))(x)
    return Concatenate()([u, xskip])

def res_unet(sz, f):
    inputs = Input(sz)
    
    ## downsample  
    e1 = bottleneck_block(inputs, f); f = int(f*2)
    e2 = res_block(e1, f, strides=2); f = int(f*2)
    e3 = res_block(e2, f, strides=2); f = int(f*2)
    e4 = res_block(e3, f, strides=2); f = int(f*2)
    _ = res_block(e4, f, strides=2)
    
    ## bottleneck
    b0 = conv_block(_, f, strides=1)
    _ = conv_block(b0, f, strides=1)
    
    ## upsample
    _ = upsamp_concat_block(_, e4)
    _ = res_block(_, f); f = int(f/2)
    
    _ = upsamp_concat_block(_, e3)
    _ = res_block(_, f); f = int(f/2)
    
    _ = upsamp_concat_block(_, e2)
    _ = res_block(_, f); f = int(f/2)
    
    _ = upsamp_concat_block(_, e1)
    _ = res_block(_, f)
    
    ## classify
    outputs = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(_)
    
    #model creation 
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

<table style="font-size: 1em; padding: 0; margin: 0;">

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #182AEB;">
    </small><br/>Optimization strategy: Dealing with class imbalance <br/> using dice loss</h1>
<br/>
<p style="border-left: 15px solid #182AEB; text-align:justify; padding: 0 10px;">
    the network tends to <b>ignore smaller classes</b>. A soft dice loss could be used to train a model. Unlike the IoU metric, the numerator is the number of correctly classified pixels, and the denominator is the total number of pixels in a class that is in both estimated and ground truth. Thus it is insensitive to the number of pixels total in each class.  
        </tr>
        </table>

In [ ]:
from tensorflow.keras.layers import Flatten

In [5]:
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = tf.reshape(tf.dtypes.cast(y_true, tf.float32), [-1])
    y_pred_f = tf.reshape(tf.dtypes.cast(y_pred, tf.float32), [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

<table style="font-size: 1em; padding: 0; margin: 0;">

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #182AEB;">
    </small><br/>Optimization strategy: Refining label images <br/> using conditional random fields</h1>
<br/>
<p style="border-left: 15px solid #182AEB; text-align:justify; padding: 0 10px;">
It is common to use some form of post-processing algorithm to refine either labels for training, or model predictions, on an image-by-image basis. A popular approach is called a fully connected or dense conditional random field or CRF. Each image is paired with a label image and the label image is refined based on the image. The CRF makes a model for the label given the image, then assesses the likelihood of each label given the CRF model. The model can be based on relative color differences between a label and the CRF model's idea of what color that label is (within a tolerance), and/or relative spatial differences (i.e. location within the image) between a given label and the CRF model's idea of the location of that label, again within a tolerance.
</p>
<p style="border-left: 15px solid #6019D6; padding: 0 10px; text-align:justify;">
    <strong style="color: #6019D6;">Tip.</strong> 
    We're using the CRF to correct a pixelwise (dense) label image. It can also be used as an 'inpainting' algorithm if only sparse areas of the image have labels. This was the approach proposed by <a href="https://www.mdpi.com/2076-3263/8/7/244">Buscombe and Ritchie (2018)</a> 
</p>
        </tr>
        </table>

Uncomment below to install the module and its dependencies:

In [47]:
#!pip install cython
#!pip install git+https://github.com/lucasb-eyer/pydensecrf.git

In [43]:
from pydensecrf import densecrf
from pydensecrf.utils import unary_from_labels
import numpy as np

ModuleNotFoundError: No module named 'pydensecrf'

The function below will refine a label image based on an undelying image, using a dense CRF. The model's hyperparameters (i.e. specified by you, not by training) are:
 
```compat_spat``` is a non-dimensional parameter that penalizes small pieces of segmentation that are spatially isolated -- it enforces more spatially consistent segmentations. Larger values means larger pieces of segmentation are allowed.

```compat_col``` is a non-dimensional parameter that penalizes pieces of segmentation that are less uniform in color -- it enforces more consistent segmentations in colorspace. Larger values means pieces of segmentation with less similar image intesity are allowed.

`theta_spat` and `theta_col` are tolerances in location and intensity, respectively. Larger values means pixel pairs can be considered to be the same class label with less similar location or intensity.

`num_iter` is the number of iterations to run the CRF model inference

In [39]:
def crf_labelrefine(input_image, predicted_labels):
    
    compat_spat=10
    compat_col=30
    theta_spat = 20
    theta_col = 80
    num_iter = 7
    
    h, w = input_image.shape[:2] #get image dimensions
    
    d = densecrf.DenseCRF2D(w, h, 2) #create a CRF object

    # For the predictions, densecrf needs 'unary potentials' which are labels (water or no water)
    predicted_unary = unary_from_labels(predicted_labels.astype('int')+1, num_classes, gt_prob= 0.51)
    
    # set the unary potentials to CRF object
    d.setUnaryEnergy(predicted_unary)

    # to add the color-independent term, where features are the locations only:
    d.addPairwiseGaussian(sxy=(theta_spat, theta_spat), compat=compat_spat, kernel=densecrf.DIAG_KERNEL,
                          normalization=densecrf.NORMALIZE_SYMMETRIC)

    input_image_uint = (input_image * 255).astype(np.uint8) #enfore unsigned 8-bit
    # to add the color-dependent term, i.e. 5-dimensional features are (x,y,r,g,b) based on the input image:    
    d.addPairwiseBilateral(sxy=(theta_col, theta_col), srgb=(5, 5, 5), rgbim=input_image_uint,
                           compat=compat_col, kernel=densecrf.DIAG_KERNEL, 
                           normalization=densecrf.NORMALIZE_SYMMETRIC)

    # Finally, we run inference to obtain the refined predictions:
    refined_predictions = np.array(d.inference(num_iter)).reshape(num_classes, h, w)
    
    # since refined_predictions will be a 2 x width x height array, 
    # each slice respresenting probability of each class (water and no water)
    # therefore we return the argmax over the zeroth dimension to return a mask
    return np.argmax(refined_predictions,axis=0)

We've seen this function before, but this time we'll need to add a line that carries out the CRF post-processing on the input (training) mask

In [23]:
def image_batch_generator(files, batch_size = 32, sz = (512, 512)):
  
  while True: # this is here because it will be called repeatedly by the training function
    
    #extract a random subset of files of length "batch_size"
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs (x) and outputs (y)
    batch_x = []
    batch_y = []
    
    #cycle through each image in the batch
    for f in batch:

        #preprocess the raw images 
        rawfile = f'nwpu_images/data/{f}'
        raw = Image.open(rawfile)
        raw = raw.resize(sz)
        raw = np.array(raw)

        #check the number of channels because some of the images are RGBA or GRAY
        if len(raw.shape) == 2:
            raw = np.stack((raw,)*3, axis=-1)

        else:
            raw = raw[:,:,0:3]
            
        #get the image dimensions, find the min dimension, then square the image off    
        nx, ny, nz = np.shape(raw)
        n = np.minimum(nx,ny)
        raw = raw[:n,:n,:] 
            
        batch_x.append(raw)
        
        #get the masks. 
        maskfile = rawfile.replace('nwpu_images','nwpu_label_images')+'_mask.jpg'
        mask = Image.open(maskfile)
        # the mask is 3-dimensional so get the max in each channel to flatten to 2D
        mask = np.max(np.array(mask.resize(sz)),axis=2)
        # water pixels are always greater than 100
        mask = (mask>200).astype('int')
        
        mask = mask[:n,:n]
        
        # use CRF to refine mask before it is used as a label
        mask = crf_labelrefine(raw, mask).squeeze()

        batch_y.append(mask)

    #preprocess a batch of images and masks 
    batch_x = np.array(batch_x)/255. #divide image by 255 to normalize
    batch_y = np.array(batch_y)
    batch_y = np.expand_dims(batch_y,3) #add singleton dimension to batch_y

    yield (batch_x, batch_y) #yield both the image and the label together

Train the model like previously. This time both input and output masks of an image will be hopefully improved using the CRF algorithm

<table style="font-size: 1em; padding: 0; margin: 0;">

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #182AEB;">
    </small><br/>Optimization strategy: <br/> using ensemble predictions</h1>
<br/>
<p style="border-left: 15px solid #182AEB; text-align:justify; padding: 0 10px;">
    It is common to train several models and average their predictions. This is called <b>ensemble</b> modeling because you are using the group of models to make a single prediction. 
</p>
<p style="border-left: 15px solid #6019D6; padding: 0 10px; text-align:justify;">
    <strong style="color: #6019D6;">Tip.</strong> 
    We have several trained models. Here we will apply them all to the same imagery and average the masks, to see if that produces a more stable estimate
</p>
        </tr>
        </table>

Define two models, compile them, and assign them the weights from two different 'h5' files saved during model training using the `load_weights` utility

In [8]:
file_id = '1BEw63yYh1Wt6Dbz85gm-U1wmi2Nyddbo'
destination = 'opt_model_5_res_dice'
if colab==1:
    download_file_from_google_drive(file_id, destination)

#s2 cloudless labels
file_id = '1QgJqz2el-9it4e9rAUUeQgWMpZbcMlKL'
destination = 'opt_model_6_res_dice2'
if colab==1:
    download_file_from_google_drive(file_id, destination)

m1 = tf.keras.models.load_model('opt_model_6_res_dice2',
                               custom_objects = {"dice_coef_loss":dice_coef_loss,
                                                "dice_coef":dice_coef})
m2 = tf.keras.models.load_model('opt_model_5_res_dice',
                               custom_objects = {"dice_coef_loss":dice_coef_loss,
                                                "dice_coef":dice_coef})
#m1 = tf.keras.models.load_model('/Users/hankmobley/Downloads/opt_model_6_res_dice2',
#                               custom_objects = {"dice_coef_loss":dice_coef_loss,
#                                                "dice_coef":dice_coef})
#m2 = tf.keras.models.load_model('/Users/hankmobley/Downloads/opt_model_5_res_dice',
#                               custom_objects = {"dice_coef_loss":dice_coef_loss,
#                                                "dice_coef":dice_coef})

In [9]:
if colab==1:
    !mkdir weights
    m1.save_weights('weights/res_dice_crf_unet2.h5')
    m2.save_weights('weights/res_dice_unet2.h5')

In [18]:
model1 = res_unet((512, 512, 3), 32)
model1.compile(optimizer = 'rmsprop', loss = dice_coef_loss, metrics = [dice_coef])
model1.load_weights('weights/res_dice_crf_unet2.h5')
#model1 = tf.model1.load_model('/Users/hankmobley/Downloads/opt_model_6_res_dice2')

model2 = res_unet((512, 512, 3), 32)
model2.compile(optimizer = 'rmsprop', loss = dice_coef_loss, metrics = [dice_coef])
model2.load_weights('weights/res_dice_unet2.h5')

Define a function for 1) getting estimates from models 1 and 2; 2) take the pixelwise maximum of the two; 3) obtain a CRF-refined estimated water mask, and 4) compute an IOU score evaluated against the real mask

In [19]:
def get_pred(x, y, model1, model2):
    #predict the mask 
    pred1 = model1.predict(np.expand_dims(x, 0))
    pred2 = model2.predict(np.expand_dims(x, 0))
    
    #mask post-processing 
    msk  = np.maximum(pred1.squeeze(), pred2.squeeze())
    # binarize
    msk[msk >= 0.5] = 1 
    msk[msk < 0.5] = 0
    
    # use CRF to refine mask before it is used as a label
    msk = crf_labelrefine(x.squeeze(), msk).squeeze()
        
    # return the prediction and the IOU score of the prediction
    return msk, mean_iou(y, msk)

Get a batch of images and labels and run the `get_pred` command on the first pair

In [46]:
#x, y = next(image_batch_generator)
x, y = next(test_generator)
ypred = get_pred(x[0], y[0], model1, model2)

NameError: name 'densecrf' is not defined

Show the label mask and the ensemble estimate side-by-side

In [ ]:
plt.subplot(121)
plt.imshow(x[0])
plt.imshow(y[0].squeeze(), cmap='gray', alpha=0.5)

plt.subplot(122)
plt.imshow(x[0])
plt.imshow(ypred.squeeze(), cmap='gray', alpha=0.5)

<table style="font-size: 1em; padding: 0; margin: 0;">
<p style="border: 1px solid #ff5733; border-left: 15px solid #ff5733; padding: 10px; text-align:justify;">
    <strong style="color: #ff5733">Deliverable</strong>  
    <br/>The deliverable for Part 5 is a jupyter notebook showing a workflow to optimize the training of a model using the NWPU-RESISC45 lake images and corresponding labels, for the purposes of estimating lake area over time at sites represented in the Sentinel-2 imagery. The notebook will also show how to refine image labels based on a CRF model using both image color and relative spatial location in the image to make predictions, using optimized tunable parameters. This will mostly test your understanding of the generic yet complex workflow of optimizing model training, and applying a model trained on one dataset to another similar dataset, for the operational purpose of developing a time-series of lake areas at critical sites. 
    </p>